In [1]:
!pip install opencv-python keras tensorflow spotipy textblob

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ------ --------------------------------- 6.0/39.5 MB 41.2 MB/s eta 0:00:01
   ------- -------------------------------- 7.1/39.5 MB 43.7 MB/s eta 0:00:01
   -------------------- ------------------- 19.9/39.5 MB 35.9 MB/s eta 0:00:01
   ---------------------------- ----------- 27.8/39.5 MB 36.7 MB/s eta 0:00:01
   ------------------------------------ --- 35.9/39.5 MB 36.8 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 35.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   --------------------------------------- 624.3/624.3 kB 24.1 MB/s eta 0:00:00


In [29]:
# Import required libraries
import cv2
import numpy as np
from keras.models import load_model
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from textblob import TextBlob
import tkinter as tk 
from tkinter import messagebox, simpledialog

SPOTIFY API

In [9]:
# Spotify API credentials
SPOTIPY_CLIENT_ID = '8a585baa3e9943e28fb1af94fcc60e07'
SPOTIPY_CLIENT_SECRET = 'c17d81613bf148f39aa5d3d437f0c64b'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                              client_secret=SPOTIPY_CLIENT_SECRET,
                                              redirect_uri=SPOTIPY_REDIRECT_URI,
                                              scope="user-library-read playlist-modify-public"))

FACE DETECTION 

In [11]:
def detect_emotion_from_face():
    # Load pre-trained emotion detection model
    model = load_model('emotion_model.h5')  # Replace with your model path
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    # Initialize webcam
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.astype('float') / 255.0
            roi = np.expand_dims(roi, axis=0)
            preds = model.predict(roi)[0]
            emotion = emotions[np.argmax(preds)]
            cv2.putText(frame, emotion, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return emotion

TEXT DETECTION

In [22]:
def detect_emotion_from_text():
    user_input = input("How are you feeling today? ")
    analysis = TextBlob(user_input)
    if analysis.sentiment.polarity > 0:
        return "Happy"
    elif analysis.sentiment.polarity < 0:
        return "Sad"
    else:
        return "Neutral"

EMOTION MAPPING 

In [15]:
def get_emotion_based_playlist(emotion):
    if emotion == "Happy":
        results = sp.recommendations(seed_genres=['pop'], limit=10, target_valence=0.8, target_energy=0.8)
    elif emotion == "Sad":
        results = sp.recommendations(seed_genres=['blues'], limit=10, target_valence=0.2, target_energy=0.3)
    elif emotion == "Angry":
        results = sp.recommendations(seed_genres=['rock'], limit=10, target_valence=0.3, target_energy=0.9)
    elif emotion == "Calm":
        results = sp.recommendations(seed_genres=['classical'], limit=10, target_valence=0.5, target_energy=0.4)
    else:
        results = sp.recommendations(seed_genres=['pop'], limit=10)

    playlist = []
    for track in results['tracks']:
        playlist.append({'name': track['name'], 'artist': track['artists'][0]['name'], 'url': track['external_urls']['spotify']})
    return playlist

In [17]:
def create_playlist(emotion, playlist):
    user_id = sp.me()['id']
    playlist_name = f"{emotion} Playlist"
    new_playlist = sp.user_playlist_create(user_id, playlist_name, public=True)
    playlist_id = new_playlist['id']

    track_uris = [sp.search(q=f"{song['name']} {song['artist']}", type='track', limit=1)['tracks']['items'][0]['uri'] for song in playlist]
    sp.playlist_add_items(playlist_id, track_uris)
    print(f"Playlist created: {new_playlist['external_urls']['spotify']}")

GUI

In [25]:
class PlaylistGeneratorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Emotion-Driven Playlist Generator")

        # Create GUI elements
        self.label = tk.Label(root, text="Welcome to the Emotion-Driven Playlist Generator!", font=("Arial", 16))
        self.label.pack(pady=20)

        self.face_button = tk.Button(root, text="Detect Emotion from Face", command=self.detect_face_emotion, font=("Arial", 14))
        self.face_button.pack(pady=10)

        self.text_button = tk.Button(root, text="Enter Emotion Manually", command=self.detect_text_emotion, font=("Arial", 14))
        self.text_button.pack(pady=10)

    def detect_face_emotion(self):
        emotion = detect_emotion_from_face()
        self.generate_playlist(emotion)

    def detect_text_emotion(self):
        emotion = detect_emotion_from_text()
        self.generate_playlist(emotion)

    def generate_playlist(self, emotion):
        playlist = get_emotion_based_playlist(emotion)
        create_playlist(emotion, playlist)

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = PlaylistGeneratorApp(root)
    root.mainloop()